In [24]:
### Migracion s3 
### Leer datps de un objeto json y analizarlos para validar si se encuentran en el json del estandar(OPEN API )

In [25]:
import glob
import json 
import polars as pl
from pathlib import Path
import pandas as pd
from pprint import pprint
from typing import Dict, Any 
from dateutil.parser import parse
from concurrent.futures import ThreadPoolExecutor


#### Valdiar datos de entrada contra esquema definida con Open-Api

#### Datos de entrada

In [26]:
ruta_datos_entrada_s3s = '../pruebas/datos_entrada/s3s/'
ruta_datos_entrada_s3p = '../pruebas/datos_entrada/s3p/'
ruta_datos_salida = '../pruebas/datos_salida/'


#### Analizar el tipo de faltas en los bulk de datos

In [43]:

def process_json_files(input_path: str, output_file: str):
    """
    Carga archivos JSON de un directorio y sus subdirectorios en un DataFrame de Polars,
    los concatena y guarda el resultado en un archivo.
    
    :param input_path: Ruta al directorio de entrada que contiene los archivos JSON.
    :param output_file: Ruta y nombre del archivo de salida.
    """
    input_path = Path(input_path)
    all_data = []
    all_columns = set()

    # Primera pasada: recopilar todas las columnas únicas y datos
    for json_file in input_path.rglob('*.json'):
        try:
            with open(json_file, 'r', encoding='utf-8') as file:
                data = json.load(file)
            for item in data:
                all_columns.update(item.keys())
                all_data.append(item)
        except Exception as e:
            print(f"Error al procesar el archivo {json_file}: {e}")

    # Crear un DataFrame con todos los datos y columnas
    if all_data:
        # Asegurarse de que todas las filas tengan todas las columnas
        for item in all_data:
            for col in all_columns:
                if col not in item:
                    item[col] = None

        final_df = pl.DataFrame(all_data)

        # Guardar el DataFrame final
        final_df.write_parquet(output_file)
        print(f"Datos guardados exitosamente en {output_file}")
        print(f"Columnas en el DataFrame final: {final_df.columns}")
    else:
        print("No se encontraron archivos JSON válidos para procesar.")

In [44]:
process_json_files(ruta_datos_entrada_s3s, ruta_datos_salida + 'datos_salida_s3s.parquet')      

Datos guardados exitosamente en ../pruebas/datos_salida/datos_salida_s3s.parquet
Columnas en el DataFrame final: ['id', 'institucionDependencia', 'servidorPublicoSancionado', 'tipoFalta', 'resolucion', 'inhabilitacion', '__v', 'autoridadSancionadora', 'causaMotivoHechos', 'documentos', 'expediente', 'fechaCaptura', 'observaciones', 'tipoSancion', 'multa']


In [34]:
### v1
#s3p_v1 = "{'id': null,'fechaCaptura': null,'expediente': null,'institucionDependencia': {'nombre': null,'siglas': null,'clave': null},'particularSancionado': {'nombreRazonSocial': null,'objetoSocial': null,'rfc': null,'tipoPersona': null,'telefono': null,'domicilioMexico': {'pais': {'valor': null,'clave': null},'entidadFederativa': {'valor': null,'clave': null},'municipio': {'valor': null,'clave': null},'codigoPostal': null,'localidad': {'valor': null,'clave': null},'vialidad': {'clave': null,'valor': null},'numeroExterior': null,'numeroInterior': null},'domicilioExtranjero': {'calle': null,'numeroExterior': null,'numeroInterior': null,'ciudadLocalidad': null,'estadoProvincia': null,'pais': {'clave': null,'valor': null},'codigoPostal': null}},'directorGeneral': {'nombres': null,'primerApellido': null,'segundoApellido': null,'curp': null},'apoderadoLegal': {'nombres': null,'primerApellido': null,'segundoApellido': null,'curp': null},'objetoContrato': null,'autoridadSancionadora': null,'tipoFalta': null,'tipoSancion': {'clave': null,'valor': null,'descripcion': null},'causaMotivoHechos': null,'acto': null,'responsableSancion': {'nombres': null,'primerApellido': null,'segundoApellido': null},'resolucion': {'sentido': null,'url': null,'fechaNotificacion': null},'multa': {'monto': null,'moneda': {'clave': null,'valor': null}},'inhabilitacion': {'plazo': null,'fechaInicial': null,'fechaFinal': null},'observaciones': null,'documentos': [{'id': null,'tipo': null,'titulo': null,'descripcion': null,'url': null,'fecha': null}]}"
#s3s_v1 = '{"id":null,"fechaCaptura":null,"expediente":null,"institucionDependencia":{"nombre":null,"siglas":null,"clave":null},"particularSancionado":{"nombreRazonSocial":null,"objetoSocial":null,"rfc":null,"tipoPersona":null,"telefono":null,"domicilioMexico":{"pais":{"valor":null,"clave":null},"entidadFederativa":{"valor":null,"clave":null},"municipio":{"valor":null,"clave":null},"codigoPostal":null,"localidad":{"valor":null,"clave":null},"vialidad":{"clave":null,"valor":null},"numeroExterior":null,"numeroInterior":null},"domicilioExtranjero":{"calle":null,"numeroExterior":null,"numeroInterior":null,"ciudadLocalidad":null,"estadoProvincia":null,"pais":{"clave":null,"valor":null},"codigoPostal":null}},"directorGeneral":{"nombres":null,"primerApellido":null,"segundoApellido":null,"curp":null},"apoderadoLegal":{"nombres":null,"primerApellido":null,"segundoApellido":null,"curp":null},"objetoContrato":null,"autoridadSancionadora":null,"tipoFalta":null,"tipoSancion":{"clave":null,"valor":null,"descripcion":null},"causaMotivoHechos":null,"acto":null,"responsableSancion":{"nombres":null,"primerApellido":null,"segundoApellido":null},"resolucion":{"sentido":null,"url":null,"fechaNotificacion":null},"multa":{"monto":null,"moneda":{"clave":null,"valor":null}},"inhabilitacion":{"plazo":null,"fechaInicial":null,"fechaFinal":null},"observaciones":null,"documentos":[{"id":null,"tipo":null,"titulo":null,"descripcion":null,"url":null,"fecha":null}]}'


In [46]:
# cargar y mostrar el archivo de salida de concatenar los datos
df_s3s = pl.read_parquet(ruta_datos_salida + 'datos_salida_s3s.parquet')
print(df.head(5))

shape: (5, 15)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ id        ┆ instituci ┆ servidorP ┆ tipoFalta ┆ … ┆ fechaCapt ┆ observaci ┆ tipoSanci ┆ multa    │
│ ---       ┆ onDepende ┆ ublicoSan ┆ ---       ┆   ┆ ura       ┆ ones      ┆ on        ┆ ---      │
│ str       ┆ ncia      ┆ cionado   ┆ struct[3] ┆   ┆ ---       ┆ ---       ┆ ---       ┆ struct[2 │
│           ┆ ---       ┆ ---       ┆           ┆   ┆ str       ┆ str       ┆ list[stru ┆ ]        │
│           ┆ struct[3] ┆ struct[5] ┆           ┆   ┆           ┆           ┆ ct[3]]    ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 6539702fe ┆ {"FISCALI ┆ {"JORGE", ┆ {"ANG","A ┆ … ┆ 2023-10-2 ┆ FALTAS    ┆ [{"I","IN ┆ null     │
│ 61a02001d ┆ A GENERAL ┆ "ALEJO    ┆ DMINISTRA ┆   ┆ 5T19:53:4 ┆ ADMINISTR ┆ HABILITAD ┆          │
│ 90f4bf    ┆ DEL       ┆ ","MIKEL" ┆ TIVA NO   ┆   ┆ 5+00:00   ┆ ATIVAS    

In [52]:
pprint(df_s3s.columns)
pprint(type(df_s3s))
pprint(df_s3s[0])

['id',
 'institucionDependencia',
 'servidorPublicoSancionado',
 'tipoFalta',
 'resolucion',
 'inhabilitacion',
 '__v',
 'autoridadSancionadora',
 'causaMotivoHechos',
 'documentos',
 'expediente',
 'fechaCaptura',
 'observaciones',
 'tipoSancion',
 'multa']
<class 'polars.dataframe.frame.DataFrame'>
shape: (1, 15)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ id        ┆ instituci ┆ servidorP ┆ tipoFalta ┆ … ┆ fechaCapt ┆ observaci ┆ tipoSanci ┆ multa    │
│ ---       ┆ onDepende ┆ ublicoSan ┆ ---       ┆   ┆ ura       ┆ ones      ┆ on        ┆ ---      │
│ str       ┆ ncia      ┆ cionado   ┆ struct[3] ┆   ┆ ---       ┆ ---       ┆ ---       ┆ struct[2 │
│           ┆ ---       ┆ ---       ┆           ┆   ┆ str       ┆ str       ┆ list[stru ┆ ]        │
│           ┆ struct[3] ┆ struct[5] ┆           ┆   ┆           ┆           ┆ ct[3]]    ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪

In [ ]:
pprint(df_s3s[0].in)